In [9]:
import pandas as pd
import numpy as np
import json

from tqdm.notebook import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
# imported = pd.read_csv('../data/old_vers/imported.csv')
# exported = pd.read_csv('../data/old_vers/exported.csv')

C:\Users\Данил\AppData\Local\Temp\ipykernel_13932\2995106456.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  imported = pd.read_csv('../data/old_vers/imported.csv')
C:\Users\Данил\AppData\Local\Temp\ipykernel_13932\2995106456.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  exported = pd.read_csv('../data/old_vers/exported.csv')


In [ ]:
imported = full_df.loc[full_df['napr'] == 'ИМ']   #Разделение на датафреймы для импорта и экспорта из общего
exported = full_df.loc[full_df['napr'] == 'ЭК']  

In [4]:
imported.columns

Index(['napr', 'period', 'nastranapr', 'tnved', 'edizm', 'Stoim', 'Netto',
       'Kol', 'Region', 'Region_s'],
      dtype='object')

In [5]:
imported = imported[['period', 'nastranapr', 'tnved', 'Stoim', 'Netto']]    #Оставляем только нужные колонки
exported = exported[['period', 'nastranapr', 'tnved', 'Stoim', 'Netto']]

In [12]:
imported.dtypes, exported.dtypes

(period        datetime64[ns]
 nastranapr            object
 tnved                 object
 Stoim                float64
 Netto                float64
 dtype: object,
 period        datetime64[ns]
 nastranapr            object
 tnved                 object
 Stoim                float64
 Netto                float64
 dtype: object)

In [8]:
def del_tochka_stoim(row):     #Приводим к типу float поля Stoim Netto
    return float(row['Stoim'].replace(',', '.'))
def del_tochka_netto(row):
    return float(row['Netto'].replace(',', '.'))

In [10]:
imported['Stoim'] = imported.parallel_apply(del_tochka_stoim, axis=1)
imported['Netto'] = imported.parallel_apply(del_tochka_netto, axis=1)

exported['Stoim'] = exported.parallel_apply(del_tochka_stoim, axis=1)
exported['Netto'] = exported.parallel_apply(del_tochka_netto, axis=1)

In [11]:
imported['period'] = pd.to_datetime(imported['period'], format='%m/%Y')
exported['period'] = pd.to_datetime(exported['period'], format='%m/%Y')

In [13]:
imported['tnved'] = imported['tnved'].astype('str')    #Приводим tnved к string
exported['tnved'] = exported['tnved'].astype('str')

In [16]:
imported = imported.loc[imported['tnved'] != 'XXXXXXXXXX']    #Удаляем неизвестные tnved
exported = exported.loc[exported['tnved'] != 'XXXXXXXXXX']

In [24]:
imported.sort_values('tnved', inplace=True)
exported.sort_values('tnved', inplace=True)

In [107]:
gr_imp = imported.groupby(['tnved', 'period']).sum()
gr_exp = exported.groupby(['tnved', 'period']).sum()

In [108]:
gr_imp.reset_index(inplace=True)
gr_exp.reset_index(inplace=True)

In [109]:
gr_imp['S/N'] = gr_imp['Stoim'] / gr_imp['Netto']    #Новый столбец для удельной стоимости за 1 кг = Цена
gr_exp['S/N'] = gr_exp['Stoim'] / gr_exp['Netto']

In [110]:
def spis(data):
    k = 0
    p = []
    t = 0
    for i, j in enumerate(data['tnved']):
        if k != j:
            k = j
            t = i
            p.append(t)
    return p  

In [111]:
spis_imp = spis(gr_imp)
spis_exp = spis(gr_exp)

In [112]:
def razn_in_spisok(lst:list, data):
    lst_razn = []
    for i in range(len(lst)-1):
        lst_razn.append(lst[i+1] - lst[i] - 1)
    lst_razn.append(data.index[-1] - lst[-1] - 1)
    return lst_razn

In [113]:
spisR_imp = razn_in_spisok(spis_imp, gr_imp)
spisR_exp = razn_in_spisok(spis_exp, gr_exp)

                       Заполняем новую колонку процентными изменениями относительно прошлых периодов для товара

In [114]:
k = spis_imp[0]
zap_imp = {}
for i, j in tqdm(zip(spis_imp[1:], spisR_imp)):
    try:
        zap_imp[i-1] = gr_imp['S/N'][k:i].pct_change(periods=j)[i-1]
    except:
        pass
    k = i

0it [00:00, ?it/s]

In [115]:
k = spis_exp[0]
zap_exp = {}
for i, j in tqdm(zip(spis_exp[1:], spisR_exp)):
    try:
        zap_exp[i-1] = gr_exp['S/N'][k:i].pct_change(periods=j)[i-1]
    except:
        pass
    k = i

0it [00:00, ?it/s]

In [116]:
gr_imp['inc'] = np.nan
gr_exp['inc'] = np.nan

In [117]:
for index, row in tqdm(gr_imp.iterrows()):
    if index in zap_imp.keys():
        gr_imp['inc'][index] = zap_imp[index]

0it [00:00, ?it/s]

C:\Users\Данил\AppData\Local\Temp\ipykernel_13932\3250029282.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_imp['inc'][index] = zap_imp[index]


In [118]:
for index, row in tqdm(gr_exp.iterrows()):
    if index in zap_exp.keys():
        gr_exp['inc'][index] = zap_exp[index]

0it [00:00, ?it/s]

C:\Users\Данил\AppData\Local\Temp\ipykernel_13932\2955583829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gr_exp['inc'][index] = zap_exp[index]


In [251]:
sorting_imp = gr_imp.groupby('tnved').mean().sort_values('inc', ascending=False)
sorting_imp.reset_index(inplace=True)
sorting_imp['First_tnved'] = sorting_imp['tnved'].apply(lambda x: x[:2])

In [252]:
sorting_exp = gr_exp.groupby('tnved').mean().sort_values('inc', ascending=False)
sorting_exp.reset_index(inplace=True)

In [254]:
lst = sorting_imp[:100]['First_tnved'].tolist()
spisok_rate = []
for i in lst:
    if i not in spisok_rate:
        spisok_rate.append(i)


In [255]:
sorting_imp.drop('First_tnved', axis=1, inplace=True)
sorting_imp['name'] = sorting_imp['tnved'].map(new_df_dict)
sorting_exp['name'] = sorting_exp['tnved'].map(new_df_dict)
sorting_imp.dropna(inplace=True)
sorting_exp.dropna(inplace=True)
sorting_imp.reset_index(drop=True, inplace=True)
sorting_exp.reset_index(drop=True, inplace=True)

In [256]:
df_imp = sorting_imp.loc[sorting_imp['tnved'].str.match(spisok_rate[0]+'\w+')]
df_imp = df_imp[:5]
for i in range(1, len(spisok_rate[:10])+1):
    df_imp_s = sorting_imp.loc[sorting_imp['tnved'].str.match(spisok_rate[i]+'\w+')]
    df_imp_s = df_imp_s[:5]
    df_imp = pd.concat([df_imp, df_imp_s])

In [257]:
df_imp = df_imp[['tnved', 'name']]

In [258]:
df_imp.reset_index(inplace=True, drop=True)

In [259]:
rec_imp = df_imp['tnved'].tolist()

In [272]:
imp_dct = {}
for index, row in tqdm(sorting_imp.iterrows()):
    imp_dct[row['tnved']] = row['inc']

0it [00:00, ?it/s]

In [274]:
exp_dct = {}
for index, row in tqdm(sorting_exp.iterrows()):
    exp_dct[row['tnved']] = row['inc']

0it [00:00, ?it/s]

In [286]:
recs = []
for i in rec_imp:
    try:
        if imp_dct.get(i) > exp_dct.get(i):
            recs.append(i)
    except:
        pass

In [288]:
recs_df = pd.DataFrame(recs, columns=['tnved'])

In [290]:
recs_df['name'] = recs_df['tnved'].map(new_df_dict)

In [295]:
recs_df.to_json('../data/for_proj/Recomendations.json')

In [297]:
sorting_imp = sorting_imp[['tnved', 'inc', 'name', 'S/N']]
sorting_exp = sorting_exp[['tnved', 'inc', 'name', 'S/N']]

In [298]:
sorting_imp.to_json('../data/for_proj/Import_sorting.json')
sorting_exp.to_json('../data/for_proj/Export_sorting.json')

In [301]:
gr_imp['name'] = gr_imp['tnved'].map(new_df_dict)
gr_exp['name'] = gr_exp['tnved'].map(new_df_dict)

In [303]:
gr_imp = gr_imp[['tnved', 'period', 'S/N', 'name']]
gr_exp = gr_exp[['tnved', 'period', 'S/N', 'name']]

In [305]:
gr_imp.to_json('../data/for_proj/ForImportStatPeriods.json')
gr_exp.to_json('../data/for_proj/ForExportStatPeriods.json')

                       СОпоставление кодов tnved - их названиям

In [139]:
# ex = pd.read_excel('../data/tnved-CIS-02.xls', usecols=['TNVED', 'FULL_TEXT'])
# ex.rename(columns={'TNVED':'tnved', 'FULL_TEXT':'name'}, inplace=True)
# ex['tnved'] = ex['tnved'].astype('str')

# cols = ['tnved', 'name']

# l = []
# for i,j in tqdm(enumerate(ex['tnved'])):
#     if len(j) == 10:
#         l.append([ex['tnved'][i], ex['name'][i]])
        
# new_df = pd.DataFrame(l, columns=cols)

# new_df_dict = {}
# for ind, znac in new_df.iterrows():
#     new_df_dict[znac['tnved']] = znac['name']

0it [00:00, ?it/s]